# Fine-tuning SAM on Satellite Images for Street Segmentation


## Considerations: 

Since the model is promptable, try to chain the model using the mask input. Maybe use a similar approach as in diffusion models


---
### What to fine-tune:
- Only Mask Decoder
- Prompt Encoder & Mask Decoder

---
### Prompt Encoder: 
SAM has an prompt encoder part that takes either a list of point queries, a list of bounding boxes or masks as input. How can we use the prompt encoder in our pipeline?

#### Training

Use as input either: 
- No input
- List of point queries: 
    - Uniformly overlay the whole image with a grid of points
    - Sample coordinates of the street pixels as point queries
- Bounding box of whole image, part where streets are located, could split up streets into multiple bounding boxes
- Masks of the street pixels, could leak ground truth information? 

#### Evaluation/Testing 

- List of point queries: 
    - Uniformly overlay the whole image with a grid of points
- No input
- Bounding box of whole image

---
### Loss function: 
- F1-Loss: https://pytorch.org/torcheval/stable/generated/torcheval.metrics.functional.multiclass_f1_score.html
---
### Learning rate:
- really small values
- reduceLROnPlateau(optimizer, 'max', patience=reduce_patience, verbose=verbose, factor=reduce_factor)
- cosine annealing


In [1]:
# written by Jannek Ulm 16.5.2024
# code was inspired by the following sources: https://github.com/bnsreenu/python_for_microscopists/blob/master/331_fine_tune_SAM_mito.ipynb

from utils.image_loading import * 
from utils.torch_device import *
from SAM_Dataset import *

device = get_torch_device()
#
print(device)


###########

sat_images = load_training_images()
sat_masks = load_groundtuth_images()


############ (FACEBOOK SAM PACKAGE)
from segment_anything import SamPredictor, sam_model_registry

model_path = "/Users/janulm/Documents/ETH/SM10/Computational Intelligence Lab/eth-cil-road-segmentation/models/sam_vit_h_4b8939.pth"
model = sam_model_registry["default"](checkpoint=model_path)
predictor = SamPredictor(model)
predictor.set_image(sat_images[0])

masks, quality, logits = predictor.predict(None, None, None,multimask_output=False)

############ (TRANSFORMERS HUGGINGFACE PACKAGE)
from transformers import SamProcessor




############







mps


UnicodeDecodeError: 'utf-8' codec can't decode byte 0x80 in position 64: invalid start byte